In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, 
                                                            postgres_host, postgres_port, postgres_db))
data = pd.read_sql_query('select * from useducation', con=engine)
engine.dispose()

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 209 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     466 non-null object
STATE                           466 non-null object
YEAR                            466 non-null int64
ENROLL                          466 non-null float64
TOTAL_REVENUE                   466 non-null float64
FEDERAL_REVENUE                 466 non-null float64
STATE_REVENUE                   466 non-null float64
LOCAL_REVENUE                   466 non-null float64
TOTAL_EXPENDITURE               466 non-null float64
INSTRUCTION_EXPENDITURE         466 non-null float64
SUPPORT_SERVICES_EXPENDITURE    466 non-null float64
OTHER_EXPENDITURE               466 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      466 non-null float64
GRADES_PK_G                     466 non-null float64
GRADES_KG_G                     466 non-null float64
GRADES_4_G                      466 non-null float64
GRADES_8_G                      466 non-null flo

In [14]:
columns = data.columns.drop(['PRIMARY_KEY', 'STATE', 'YEAR','AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE',
       'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'])
states = data.STATE.unique()

for s in states:
    data.loc[data['STATE']==s, columns] = data.loc[data['STATE']==s, columns].interpolate()
    
data.dropna(inplace=True)
columns

Index(['ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE',
       'LOCAL_REVENUE', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G'],
      dtype='object')

In [25]:
data['total_4'] = data['GRADES_4_G']*(data['AVG_MATH_4_SCORE'] + data['AVG_READING_4_SCORE'])
data['total_8'] = data['GRADES_8_G']*(data['AVG_MATH_8_SCORE'] + data['AVG_READING_8_SCORE'])

data['new_score'] = (data['total_4']/2 + data['total_8']/2)/(data['GRADES_4_G'] + data['GRADES_8_G'])

data[['new_score','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE', 
     'OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE' ]].corr()

#  the instruction_expenditure seems correlated with the new_score most

,new_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
new_score,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


In [36]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [110]:
X = data[['INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE', 
     'OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE' ]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=4)
data['pca_1'] = sklearn_pca.fit_transform(X)


print(sklearn_pca.explained_variance_ratio_)


[0.9430175  0.03343929 0.02005189 0.00349132]


In [111]:
data[['new_score','pca_1','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE', 
     'OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']].corr()

# the new_score is correlated better with the instruction_expenditure than others
# pca works best when the correlation betwwen different variables are less than 0.8, in this case the correlation between different
# expenditure are very strong

,new_score,pca_1,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
new_score,1.000000,0.044596,0.087386,0.079002,0.010959,-0.005389
pca_1,0.044596,1.000000,0.968606,0.988345,0.971343,0.955792
INSTRUCTION_EXPENDITURE,0.087386,0.968606,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.884798,0.911245,0.918188,1.000000
